# 작업 내용
- 1. countvectorizer 활용 cosine유사도 분석
- 2. tf-idf 활용 cosine유사도 분석
- 로직설명
- 독자-독자 : 취향 유사도
- 작가-작가 : 분야 유사도
- 독자-작가 : 취향 유사도
- 독자 1명이 읽은 글의 전체 태그를 말뭉치로 만듦
- 작가 1명이 작성한 글의 전체 태그를 말뭉치로 만듦
- 말뭉치를 형태소로 분석
- vectorize
    - 중복 제거 (count_vectorizer 사용)
    - 중복 비제거 (tf-idf 사용)

In [9]:
from konlpy.tag import Kkma
import matplotlib.pyplot as plt
from tqdm import tqdm
from pandas import Panel
from sklearn.feature_extraction.text import TfidfVectorizer

C:\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  after removing the cwd from sys.path.


In [ ]:
from Brunch_data import *

In [3]:
meta = pd.read_json('C:/Users/eunice/dss_workspace/MLproject/data/metadata.json', lines=True)
read_file_lst = glob.glob('C:/Users/eunice/dss_workspace/MLproject/data/read\*')
maga = pd.read_json('C:/Users/eunice/dss_workspace/MLproject/data/magazine.json', lines=True)
us = pd.read_json('C:/Users/eunice/dss_workspace/MLproject/data/users.json', lines=True)

In [4]:
a = Data(meta, read_file_lst, maga, us)
me_df, r_df, mag_df, us_df = a.data_preprocessing()
# 2/15 - 3/31 train 데이터
train = a.train_data()

KeyboardInterrupt: 

In [ ]:
# article_id, keyword_list
key = a.train_keyword_list()

In [6]:
# readers별 article list와 articlle count가 있는 df :ra
ra_list_df = Read_article_outline_remove(train)
ra = ra_list_df.read_article_list()
# up = ra_list_df.upper_fence_remove()
# 5 이상 upper fence 밑으로 정리
# t3 = article_count_division_3(up)

In [7]:
upper_fence_df = ra[(ra['article_id_count']>=9) & (ra['article_id_count']<=64)]

In [107]:
upper_fence_df

,readers_id,article_list,article_id_count
11,#0013c35c789f0936654ba06389ea75cb,"[@dotkvk_9, @dotkvk_21, @dotkvk_10, @dotkvk_19...",11
22,#00222a855547061a7fd00e404cb96148,"[@kimssibang_18, @uppity_34, @julien_174, @und...",11
33,#002cedb579c73f021b86f8b12d8e35d8,"[@prestigegorilla_364, @masism_417, @masism_42...",11
34,#002f5cac6b2c802ec323d3d3424a71cf,"[@mangmangdylujz_40, @lazytraveler03_33, @pere...",23
40,#00339d09d3f964d88ae3e57a81803539,"[@heykwak_86, @reinastory_1, @wikitree_3239, @...",35
...,...,...,...
64646,#fff10b2ae2f6e5a121a0adea7062898d,"[@tamarorim_137, @myseol_3, @erin823_31, @suha...",9
64649,#fff243d06ee500cd9af1fdfe7886bac7,"[@naturalhealer_46, @lovebook_102, @lovebook_1...",52
64655,#fff892683009fe46182af61b859d2413,"[@yoonjikwon_211, @tenbody_1683, @brunchqxk5_2...",18
64661,#fffca5f83f8b7145b3102aec2e0934dc,"[@kkangnpo_99, @sterdam_971, @merryseo_79, @ga...",10


# 독자-독자 유사도 구하기
- 독자별 주로 읽은 글의 키워드 정리하기
- 1. train2 데이터와 read데이터의 독자 정보 merge
- 2. train2 데이터에서 독자 id기준 groupby/ keyword list sum
- 3. 키워드 중복 제거
- 4. 키워드 형태소 분석

In [8]:
upper_fence_df.drop(['article_list', 'article_id_count'], axis=1, inplace=True)

In [9]:
# 1. train 데이터와 upper_fence_df데이터의 독자 정보 merge (설정 기간동안 작성된 article에 대해서만 병합)
train2 = train.merge(upper_fence_df, how='inner', on='readers_id')
train2

,magazine_id,author_id,keyword_list,article_id,type,reg_dt,read_dt,read_hr,readers_id
0,30566,@maama,"[스트레스, 상황, 마음]",@maama_314,매거진,2019-02-17,20190217,14,#49ecf492599e23e022ec1a6d8aeb495a
1,30566,@maama,"[스트레스, 상황, 마음]",@maama_314,매거진,2019-02-17,20190217,14,#49ecf492599e23e022ec1a6d8aeb495a
2,2749,@kangsunseng,"[자랑질, 인스타그램, 페이스북]",@kangsunseng_843,매거진,2019-02-16,20190216,22,#49ecf492599e23e022ec1a6d8aeb495a
3,2749,@kangsunseng,"[자랑질, 인스타그램, 페이스북]",@kangsunseng_843,매거진,2019-02-16,20190216,22,#49ecf492599e23e022ec1a6d8aeb495a
4,2749,@kangsunseng,"[자랑질, 인스타그램, 페이스북]",@kangsunseng_843,매거진,2019-02-16,20190217,14,#49ecf492599e23e022ec1a6d8aeb495a
...,...,...,...,...,...,...,...,...,...
451681,38478,@999999999,"[일기, 엄마, 딸]",@999999999_128,매거진,2019-02-28,20190228,21,#942f8622567491dd64706a6e714a83c5
451682,33360,@miu,"[카메라, 유튜브, 에세이]",@miu_415,매거진,2019-02-19,20190221,09,#942f8622567491dd64706a6e714a83c5
451683,33360,@miu,"[카메라, 유튜브, 에세이]",@miu_415,매거진,2019-02-19,20190221,09,#942f8622567491dd64706a6e714a83c5
451684,0,@onionsoup,"[제과, 디저트, 공감에세이]",@onionsoup_9,개인,2019-02-15,20190219,11,#942f8622567491dd64706a6e714a83c5


In [110]:
# 1. train 데이터와 upper_fence_df데이터의 독자 정보 merge (설정 기간동안 작성된 article에 대해서만 병합)
train_key_sum = train2.groupby('readers_id')['keyword_list'].agg(sum)
train_key_sum

readers_id
#0013c35c789f0936654ba06389ea75cb    [육아, 결혼, 시어머니, 육아, 결혼, 시어머니, 결혼, 육아, 시어머니, 시어머...
#00222a855547061a7fd00e404cb96148    [음식, 일본여행, 추천, 교실영어, English, 문장, 이력서, 면접, 취업,...
#002cedb579c73f021b86f8b12d8e35d8    [호텔, 서울홍대, 여행, 호텔, 서울홍대, 여행, 세계정상, 정상, 맥주, 세계정...
#002f5cac6b2c802ec323d3d3424a71cf    [연애, 감정, 사랑, 연애, 감정, 사랑, 남편, 신혼일기, 신혼, 여행, 광양,...
#00339d09d3f964d88ae3e57a81803539    [베이킹소다, 청소, 봄맞이, 결혼, 부부, 사랑, 방콕, 태국, 야시장, 인생, ...
                                                           ...                        
#fff10b2ae2f6e5a121a0adea7062898d    [건강검진, 갑상선암, 건강, 건강검진, 갑상선암, 건강, 건강검진, 갑상선암, 건...
#fff243d06ee500cd9af1fdfe7886bac7    [아침생각, 생각, 완벽, 아침생각, 생각, 완벽, 아침생각, 생각, 완벽, 사랑,...
#fff892683009fe46182af61b859d2413    [스트레칭, 운동, 척추, 스트레칭, 운동, 척추, 고양이, 길고양이, 고양이사진,...
#fffca5f83f8b7145b3102aec2e0934dc    [여행, 국내여행, 골든리트리버, 여행, 국내여행, 골든리트리버, 경영, 디자인, ...
#fffe67ecc0056dd26ae00511957c5a2b    [정리, 용돈, 재활용, 월급, 직장, 사장, 육아, 결혼, 시어머니, 일본, 손수...
Name: keyword_list, Length: 1202

In [111]:
train_key_sum = train_key_sum.reset_index()

In [112]:
len(train_key_sum)

12022

In [113]:
# 독자별 주로 읽은 글의 키워드 
train_key_sum = train_key_sum[train_key_sum['keyword_list'].apply(lambda x: len(x))!=0]
train_key_sum

,readers_id,keyword_list
0,#0013c35c789f0936654ba06389ea75cb,"[육아, 결혼, 시어머니, 육아, 결혼, 시어머니, 결혼, 육아, 시어머니, 시어머..."
1,#00222a855547061a7fd00e404cb96148,"[음식, 일본여행, 추천, 교실영어, English, 문장, 이력서, 면접, 취업,..."
2,#002cedb579c73f021b86f8b12d8e35d8,"[호텔, 서울홍대, 여행, 호텔, 서울홍대, 여행, 세계정상, 정상, 맥주, 세계정..."
3,#002f5cac6b2c802ec323d3d3424a71cf,"[연애, 감정, 사랑, 연애, 감정, 사랑, 남편, 신혼일기, 신혼, 여행, 광양,..."
4,#00339d09d3f964d88ae3e57a81803539,"[베이킹소다, 청소, 봄맞이, 결혼, 부부, 사랑, 방콕, 태국, 야시장, 인생, ..."
...,...,...
12017,#fff10b2ae2f6e5a121a0adea7062898d,"[건강검진, 갑상선암, 건강, 건강검진, 갑상선암, 건강, 건강검진, 갑상선암, 건..."
12018,#fff243d06ee500cd9af1fdfe7886bac7,"[아침생각, 생각, 완벽, 아침생각, 생각, 완벽, 아침생각, 생각, 완벽, 사랑,..."
12019,#fff892683009fe46182af61b859d2413,"[스트레칭, 운동, 척추, 스트레칭, 운동, 척추, 고양이, 길고양이, 고양이사진,..."
12020,#fffca5f83f8b7145b3102aec2e0934dc,"[여행, 국내여행, 골든리트리버, 여행, 국내여행, 골든리트리버, 경영, 디자인, ..."


In [114]:
# 중복제거 명령 (countvectorizer에서 사용)
tqdm.pandas()
# k_list = reader_keyword.progress_apply(lambda x: list(set(x)))

In [115]:
# 5. 키워드 형태소 분석 

In [10]:
kkma = Kkma()

In [11]:
def morphs_analysis(data):
    ls = []
    for x in range(len(data)):
        ls.append(kkma.morphs(data[x]))
    return list(chain.from_iterable(ls))

In [118]:
train_key_sum['morphs_list'] = train_key_sum['keyword_list'].progress_apply(lambda x: morphs_analysis(x))



  0%|          | 0/12022 [00:00<?, ?it/s]

  0%|          | 2/12022 [00:00<34:28,  5.81it/s]

  0%|          | 3/12022 [00:00<39:07,  5.12it/s]

  0%|          | 4/12022 [00:01<52:48,  3.79it/s]

  0%|          | 5/12022 [00:06<6:26:53,  1.93s/it]

  0%|          | 6/12022 [00:07<5:32:53,  1.66s/it]

  0%|          | 7/12022 [00:08<4:58:56,  1.49s/it]

  0%|          | 8/12022 [00:09<3:37:01,  1.08s/it]

  0%|          | 9/12022 [00:10<3:34:45,  1.07s/it]

  0%|          | 10/12022 [00:12<4:33:33,  1.37s/it]

  0%|          | 11/12022 [00:12<3:23:17,  1.02s/it]

  0%|          | 12/12022 [00:16<6:34:01,  1.97s/it]

  0%|          | 13/12022 [00:16<4:43:33,  1.42s/it]

  0%|          | 14/12022 [00:19<6:25:59,  1.93s/it]

  0%|          | 16/12022 [00:20<5:04:46,  1.52s/it]

  0%|          | 17/12022 [00:23<5:35:37,  1.68s/it]

  0%|          | 18/12022 [00:23<4:01:42,  1.21s/it]

  0%|          | 19/12022 [00:24<3:44:51,  1.12s/it]

  0%|          | 20/12022 [00:24<2:44:19,  1.22it/s

  3%|▎         | 317/12022 [05:03<3:10:34,  1.02it/s]

  3%|▎         | 318/12022 [05:04<3:17:39,  1.01s/it]

  3%|▎         | 320/12022 [05:04<2:21:59,  1.37it/s]

  3%|▎         | 322/12022 [05:05<2:07:49,  1.53it/s]

  3%|▎         | 323/12022 [05:06<2:40:01,  1.22it/s]

  3%|▎         | 324/12022 [05:08<3:49:24,  1.18s/it]

  3%|▎         | 325/12022 [05:08<2:48:10,  1.16it/s]

  3%|▎         | 326/12022 [05:09<2:51:41,  1.14it/s]

  3%|▎         | 327/12022 [05:11<4:03:43,  1.25s/it]

  3%|▎         | 328/12022 [05:12<3:56:37,  1.21s/it]

  3%|▎         | 329/12022 [05:13<3:45:10,  1.16s/it]

  3%|▎         | 330/12022 [05:16<4:49:35,  1.49s/it]

  3%|▎         | 332/12022 [05:17<3:52:25,  1.19s/it]

  3%|▎         | 333/12022 [05:18<3:43:13,  1.15s/it]

  3%|▎         | 334/12022 [05:18<2:43:13,  1.19it/s]

  3%|▎         | 336/12022 [05:19<2:26:10,  1.33it/s]

  3%|▎         | 338/12022 [05:20<2:11:25,  1.48it/s]

  3%|▎         | 339/12022 [05:23<4:43:31,  1.46s/it]

  3%|▎    

  5%|▌         | 633/12022 [09:44<2:33:03,  1.24it/s]

  5%|▌         | 635/12022 [09:45<2:19:54,  1.36it/s]

  5%|▌         | 636/12022 [09:48<3:37:08,  1.14s/it]

  5%|▌         | 637/12022 [09:48<2:39:54,  1.19it/s]

  5%|▌         | 638/12022 [09:49<2:47:49,  1.13it/s]

  5%|▌         | 639/12022 [09:49<2:03:18,  1.54it/s]

  5%|▌         | 640/12022 [09:51<3:43:59,  1.18s/it]

  5%|▌         | 641/12022 [09:52<3:43:22,  1.18s/it]

  5%|▌         | 642/12022 [09:53<2:44:14,  1.15it/s]

  5%|▌         | 643/12022 [09:53<2:51:39,  1.10it/s]

  5%|▌         | 644/12022 [09:56<4:10:18,  1.32s/it]

  5%|▌         | 645/12022 [09:56<3:02:46,  1.04it/s]

  5%|▌         | 646/12022 [09:59<5:15:35,  1.66s/it]

  5%|▌         | 648/12022 [10:02<5:07:46,  1.62s/it]

  5%|▌         | 650/12022 [10:03<4:07:54,  1.31s/it]

  5%|▌         | 652/12022 [10:04<2:58:33,  1.06it/s]

  5%|▌         | 653/12022 [10:05<2:56:42,  1.07it/s]

  5%|▌         | 654/12022 [10:05<2:09:55,  1.46it/s]

  5%|▌    

  8%|▊         | 958/12022 [14:28<4:20:27,  1.41s/it]

  8%|▊         | 959/12022 [14:31<5:02:51,  1.64s/it]

  8%|▊         | 960/12022 [14:31<3:38:59,  1.19s/it]

  8%|▊         | 962/12022 [14:32<2:57:55,  1.04it/s]

  8%|▊         | 963/12022 [14:33<3:07:49,  1.02s/it]

  8%|▊         | 964/12022 [14:36<5:07:26,  1.67s/it]

  8%|▊         | 966/12022 [14:37<4:04:35,  1.33s/it]

  8%|▊         | 967/12022 [14:39<4:59:12,  1.62s/it]

  8%|▊         | 968/12022 [14:41<5:30:25,  1.79s/it]

  8%|▊         | 969/12022 [14:42<4:43:44,  1.54s/it]

  8%|▊         | 971/12022 [14:43<3:45:27,  1.22s/it]

  8%|▊         | 973/12022 [14:43<2:41:02,  1.14it/s]

  8%|▊         | 974/12022 [14:44<2:46:51,  1.10it/s]

  8%|▊         | 975/12022 [14:46<2:55:46,  1.05it/s]

  8%|▊         | 977/12022 [14:46<2:09:42,  1.42it/s]

  8%|▊         | 978/12022 [14:47<2:29:18,  1.23it/s]

  8%|▊         | 980/12022 [14:49<2:47:20,  1.10it/s]

  8%|▊         | 981/12022 [14:49<2:03:04,  1.50it/s]

  8%|▊    

 11%|█         | 1280/12022 [19:23<1:19:09,  2.26it/s]

 11%|█         | 1281/12022 [19:24<1:49:35,  1.63it/s]

 11%|█         | 1282/12022 [19:25<2:20:23,  1.27it/s]

 11%|█         | 1283/12022 [19:27<3:33:45,  1.19s/it]

 11%|█         | 1285/12022 [19:29<3:00:32,  1.01s/it]

 11%|█         | 1286/12022 [19:31<4:00:04,  1.34s/it]

 11%|█         | 1287/12022 [19:32<3:51:07,  1.29s/it]

 11%|█         | 1288/12022 [19:32<2:49:11,  1.06it/s]

 11%|█         | 1289/12022 [19:38<7:28:33,  2.51s/it]

 11%|█         | 1290/12022 [19:40<7:11:18,  2.41s/it]

 11%|█         | 1291/12022 [19:41<5:52:08,  1.97s/it]

 11%|█         | 1292/12022 [19:41<4:13:50,  1.42s/it]

 11%|█         | 1294/12022 [19:44<3:54:28,  1.31s/it]

 11%|█         | 1295/12022 [19:45<3:36:32,  1.21s/it]

 11%|█         | 1296/12022 [19:46<3:27:31,  1.16s/it]

 11%|█         | 1297/12022 [19:46<2:33:37,  1.16it/s]

 11%|█         | 1298/12022 [19:46<1:54:08,  1.57it/s]

 11%|█         | 1299/12022 [19:47<2:18:44,  1.2

...

 87%|████████▋ | 10508/12022 [2:41:16<20:08,  1.25it/s]

 87%|████████▋ | 10510/12022 [2:41:17<17:38,  1.43it/s]

 87%|████████▋ | 10511/12022 [2:41:17<13:12,  1.91it/s]

 87%|████████▋ | 10512/12022 [2:41:18<17:14,  1.46it/s]

 87%|████████▋ | 10513/12022 [2:41:19<19:24,  1.30it/s]

 87%|████████▋ | 10514/12022 [2:41:19<14:28,  1.74it/s]

 87%|████████▋ | 10515/12022 [2:41:20<18:24,  1.37it/s]

 87%|████████▋ | 10516/12022 [2:41:22<21:16,  1.18it/s]

 87%|████████▋ | 10517/12022 [2:41:23<23:16,  1.08it/s]

 87%|████████▋ | 10518/12022 [2:41:25<32:17,  1.29s/it]

 87%|████████▋ | 10519/12022 [2:41:26<32:22,  1.29s/it]

 88%|████████▊ | 10520/12022 [2:41:27<29:35,  1.18s/it]

 88%|████████▊ | 10521/12022 [2:41:27<21:32,  1.16it/s]

 88%|████████▊ | 10523/12022 [2:41:28<19:10,  1.30it/s]

 88%|████████▊ | 10524/12022 [2:41:29<21:15,  1.17it/s]

 88%|████████▊ | 10526/12022 [2:41:30<19:24,  1.28it/s]

 88%|████████▊ | 10527/12022 [2:41:31<21:08,  1.18it/s]

 88%|████████▊ | 10528/12022 [2

 90%|█████████ | 10823/12022 [2:45:45<15:22,  1.30it/s]

 90%|█████████ | 10824/12022 [2:45:45<12:08,  1.64it/s]

 90%|█████████ | 10826/12022 [2:45:48<15:54,  1.25it/s]

 90%|█████████ | 10827/12022 [2:45:48<11:57,  1.67it/s]

 90%|█████████ | 10829/12022 [2:45:50<15:06,  1.32it/s]

 90%|█████████ | 10830/12022 [2:45:53<24:53,  1.25s/it]

 90%|█████████ | 10831/12022 [2:45:54<23:49,  1.20s/it]

 90%|█████████ | 10832/12022 [2:45:54<17:31,  1.13it/s]

 90%|█████████ | 10833/12022 [2:45:55<17:27,  1.13it/s]

 90%|█████████ | 10834/12022 [2:45:55<13:08,  1.51it/s]

 90%|█████████ | 10836/12022 [2:45:56<11:56,  1.65it/s]

 90%|█████████ | 10837/12022 [2:45:57<14:26,  1.37it/s]

 90%|█████████ | 10838/12022 [2:45:57<10:51,  1.82it/s]

 90%|█████████ | 10839/12022 [2:45:57<08:11,  2.41it/s]

 90%|█████████ | 10840/12022 [2:45:58<11:23,  1.73it/s]

 90%|█████████ | 10841/12022 [2:45:58<08:58,  2.19it/s]

 90%|█████████ | 10842/12022 [2:45:59<11:35,  1.70it/s]

 90%|█████████ | 10843/12022 [2

 92%|█████████▏| 11118/12022 [2:50:37<13:44,  1.10it/s]

 92%|█████████▏| 11119/12022 [2:50:38<14:10,  1.06it/s]

 92%|█████████▏| 11120/12022 [2:50:38<10:34,  1.42it/s]

 93%|█████████▎| 11121/12022 [2:50:40<12:34,  1.19it/s]

 93%|█████████▎| 11122/12022 [2:50:42<17:47,  1.19s/it]

 93%|█████████▎| 11123/12022 [2:50:42<13:08,  1.14it/s]

 93%|█████████▎| 11124/12022 [2:50:43<13:32,  1.11it/s]

 93%|█████████▎| 11126/12022 [2:50:45<14:53,  1.00it/s]

 93%|█████████▎| 11127/12022 [2:50:48<24:18,  1.63s/it]

 93%|█████████▎| 11128/12022 [2:50:49<21:40,  1.45s/it]

 93%|█████████▎| 11129/12022 [2:50:51<20:31,  1.38s/it]

 93%|█████████▎| 11130/12022 [2:50:52<18:47,  1.26s/it]

 93%|█████████▎| 11131/12022 [2:50:52<13:52,  1.07it/s]

 93%|█████████▎| 11132/12022 [2:50:53<14:38,  1.01it/s]

 93%|█████████▎| 11133/12022 [2:50:54<14:10,  1.04it/s]

 93%|█████████▎| 11134/12022 [2:50:54<10:42,  1.38it/s]

 93%|█████████▎| 11135/12022 [2:50:55<12:36,  1.17it/s]

 93%|█████████▎| 11137/12022 [2

 95%|█████████▌| 11431/12022 [2:55:04<14:13,  1.44s/it]

 95%|█████████▌| 11432/12022 [2:55:05<12:50,  1.31s/it]

 95%|█████████▌| 11433/12022 [2:55:06<11:30,  1.17s/it]

 95%|█████████▌| 11435/12022 [2:55:07<09:49,  1.00s/it]

 95%|█████████▌| 11436/12022 [2:55:12<19:17,  1.98s/it]

 95%|█████████▌| 11437/12022 [2:55:13<17:08,  1.76s/it]

 95%|█████████▌| 11438/12022 [2:55:14<14:59,  1.54s/it]

 95%|█████████▌| 11439/12022 [2:55:14<10:55,  1.12s/it]

 95%|█████████▌| 11440/12022 [2:55:15<10:28,  1.08s/it]

 95%|█████████▌| 11441/12022 [2:55:16<10:52,  1.12s/it]

 95%|█████████▌| 11442/12022 [2:55:17<10:31,  1.09s/it]

 95%|█████████▌| 11443/12022 [2:55:18<10:28,  1.09s/it]

 95%|█████████▌| 11444/12022 [2:55:18<07:41,  1.25it/s]

 95%|█████████▌| 11445/12022 [2:55:19<08:08,  1.18it/s]

 95%|█████████▌| 11446/12022 [2:55:20<08:50,  1.09it/s]

 95%|█████████▌| 11448/12022 [2:55:22<07:44,  1.24it/s]

 95%|█████████▌| 11449/12022 [2:55:22<05:44,  1.67it/s]

 95%|█████████▌| 11450/12022 [2

 98%|█████████▊| 11745/12022 [2:59:45<06:18,  1.37s/it]

 98%|█████████▊| 11747/12022 [2:59:49<06:41,  1.46s/it]

 98%|█████████▊| 11748/12022 [2:59:51<07:38,  1.67s/it]

 98%|█████████▊| 11749/12022 [2:59:53<08:26,  1.86s/it]

 98%|█████████▊| 11750/12022 [2:59:53<06:06,  1.35s/it]

 98%|█████████▊| 11751/12022 [2:59:55<05:47,  1.28s/it]

 98%|█████████▊| 11752/12022 [2:59:56<05:20,  1.19s/it]

 98%|█████████▊| 11753/12022 [2:59:56<03:57,  1.13it/s]

 98%|█████████▊| 11754/12022 [2:59:58<05:43,  1.28s/it]

 98%|█████████▊| 11755/12022 [3:00:00<06:48,  1.53s/it]

 98%|█████████▊| 11756/12022 [3:00:00<04:55,  1.11s/it]

 98%|█████████▊| 11759/12022 [3:00:00<03:28,  1.26it/s]

 98%|█████████▊| 11760/12022 [3:00:01<03:35,  1.21it/s]

 98%|█████████▊| 11762/12022 [3:00:02<03:14,  1.34it/s]

 98%|█████████▊| 11764/12022 [3:00:03<02:59,  1.44it/s]

 98%|█████████▊| 11765/12022 [3:00:05<03:33,  1.21it/s]

 98%|█████████▊| 11766/12022 [3:00:06<03:47,  1.12it/s]

 98%|█████████▊| 11767/12022 [3

In [119]:
train_key_sum

,readers_id,keyword_list,morphs_list
0,#0013c35c789f0936654ba06389ea75cb,"[육아, 결혼, 시어머니, 육아, 결혼, 시어머니, 결혼, 육아, 시어머니, 시어머...","[육아, 결혼, 시어머니, 육아, 결혼, 시어머니, 결혼, 육아, 시어머니, 시어머..."
1,#00222a855547061a7fd00e404cb96148,"[음식, 일본여행, 추천, 교실영어, English, 문장, 이력서, 면접, 취업,...","[음식, 일본, 여행, 추천, 교실, 영어, English, 문장, 이력서, 면접,..."
2,#002cedb579c73f021b86f8b12d8e35d8,"[호텔, 서울홍대, 여행, 호텔, 서울홍대, 여행, 세계정상, 정상, 맥주, 세계정...","[호텔, 서울, 홍, 대, 여행, 호텔, 서울, 홍, 대, 여행, 세계, 정상, 정..."
3,#002f5cac6b2c802ec323d3d3424a71cf,"[연애, 감정, 사랑, 연애, 감정, 사랑, 남편, 신혼일기, 신혼, 여행, 광양,...","[연애, 감정, 사랑, 연애, 감정, 사랑, 남편, 신혼, 일기, 신혼, 여행, 광..."
4,#00339d09d3f964d88ae3e57a81803539,"[베이킹소다, 청소, 봄맞이, 결혼, 부부, 사랑, 방콕, 태국, 야시장, 인생, ...","[베이, 킹, 소다, 청소, 봄맞이, 결혼, 부부, 사랑, 방, 콕, 태국, 야시장..."
...,...,...,...
12017,#fff10b2ae2f6e5a121a0adea7062898d,"[건강검진, 갑상선암, 건강, 건강검진, 갑상선암, 건강, 건강검진, 갑상선암, 건...","[건강, 검진, 갑상선, 암, 건강, 건강, 검진, 갑상선, 암, 건강, 건강, 검..."
12018,#fff243d06ee500cd9af1fdfe7886bac7,"[아침생각, 생각, 완벽, 아침생각, 생각, 완벽, 아침생각, 생각, 완벽, 사랑,...","[아침, 생각, 생각, 완벽, 아침, 생각, 생각, 완벽, 아침, 생각, 생각, 완..."
12019,#fff892683009fe46182af61b859d2413,"[스트레칭, 운동, 척추, 스트레칭, 운동, 척추, 고양이, 길고양이, 고양이사진,...","[스트레칭, 운동, 척추, 스트레칭, 운동, 척추, 고양이, 길, 고양이, 고양이,..."
12020,#fffca5f83f8b7145b3102aec2e0934dc,"[여행, 국내여행, 골든리트리버, 여행, 국내여행, 골든리트리버, 경영, 디자인, ...","[여행, 국내, 여행, 골든, 리, 트리, 벌, 여행, 국내, 여행, 골든, 리, ..."


In [3]:
# 중복제거 전 형태소분석 결과 저장하기
train_key_sum.to_csv('reader_keyword_list_mporphs_duplicates.csv', encoding='utf-8')

In [163]:
# 형태소 분석 완료
# reader_klist_df.to_csv('kewordlist_morphs.csv', encoding='utf-8')

## Tfidf vectorizer 구성

In [103]:
train_key_sum = pd.read_csv('./ml-repo-4/reader_keyword_list_mporphs_duplicates.csv', encoding='utf-8', index_col=[0]).reset_index(drop=True)

In [104]:
train_key_sum['morphs_list'] = train_key_sum['morphs_list'].apply(lambda x: str(x).replace('[', '').replace(']','').replace("'", ''))

In [105]:
train_key_sum = train_key_sum[train_key_sum['keyword_list'].apply(lambda x: len(x))!=0]
train_key_sum

,readers_id,keyword_list,morphs_list
0,#0013c35c789f0936654ba06389ea75cb,"['육아', '결혼', '시어머니', '육아', '결혼', '시어머니', '결혼',...","육아, 결혼, 시어머니, 육아, 결혼, 시어머니, 결혼, 육아, 시어머니, 시어머니..."
1,#00222a855547061a7fd00e404cb96148,"['음식', '일본여행', '추천', '교실영어', 'English', '문장', ...","음식, 일본, 여행, 추천, 교실, 영어, English, 문장, 이력서, 면접, ..."
2,#002cedb579c73f021b86f8b12d8e35d8,"['호텔', '서울홍대', '여행', '호텔', '서울홍대', '여행', '세계정상...","호텔, 서울, 홍, 대, 여행, 호텔, 서울, 홍, 대, 여행, 세계, 정상, 정상..."
3,#002f5cac6b2c802ec323d3d3424a71cf,"['연애', '감정', '사랑', '연애', '감정', '사랑', '남편', '신혼...","연애, 감정, 사랑, 연애, 감정, 사랑, 남편, 신혼, 일기, 신혼, 여행, 광,..."
4,#00339d09d3f964d88ae3e57a81803539,"['베이킹소다', '청소', '봄맞이', '결혼', '부부', '사랑', '방콕',...","베이, 킹, 소다, 청소, 봄맞이, 결혼, 부부, 사랑, 방, 콕, 태국, 야시장,..."
...,...,...,...
12017,#fff10b2ae2f6e5a121a0adea7062898d,"['건강검진', '갑상선암', '건강', '건강검진', '갑상선암', '건강', '...","건강, 검진, 갑상선, 암, 건강, 건강, 검진, 갑상선, 암, 건강, 건강, 검진..."
12018,#fff243d06ee500cd9af1fdfe7886bac7,"['아침생각', '생각', '완벽', '아침생각', '생각', '완벽', '아침생각...","아침, 생각, 생각, 완벽, 아침, 생각, 생각, 완벽, 아침, 생각, 생각, 완벽..."
12019,#fff892683009fe46182af61b859d2413,"['스트레칭', '운동', '척추', '스트레칭', '운동', '척추', '고양이'...","스트레칭, 운동, 척추, 스트레칭, 운동, 척추, 고양이, 길, 고양이, 고양이, ..."
12020,#fffca5f83f8b7145b3102aec2e0934dc,"['여행', '국내여행', '골든리트리버', '여행', '국내여행', '골든리트리버...","여행, 국내, 여행, 골든, 리, 트리, 벌, 여행, 국내, 여행, 골든, 리, 트..."


In [106]:
tfidf_vectorizer = TfidfVectorizer()

In [107]:
# 저장파일 불러왔을 때 활용
tqdm.pandas()
# train_key_sum["keyword_list_literal"] = train_key_sum['morphs_list'].progress_apply(lambda x: ("").join(x))
tf_vec = tfidf_vectorizer.fit_transform(train_key_sum['morphs_list'])
print(tf_vec)

C:\anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


  (0, 2596)	0.08892226356721587
  (0, 2493)	0.7681460744707492
  (0, 285)	0.4326307307899682
  (0, 3154)	0.46354621138932706
  (1, 2530)	0.1108507957519865
  (1, 2535)	0.10660355469600799
  (1, 717)	0.11489996573920543
  (1, 2405)	0.2888101083568638
  (1, 220)	0.45439651793448177
  (1, 3369)	0.06282610806441084
  (1, 1351)	0.09543601466677015
  (1, 2450)	0.07668297764741876
  (1, 1888)	0.12762453906603943
  (1, 2999)	0.0641434745525184
  (1, 2956)	0.20192143601667717
  (1, 4065)	0.20260131208375642
  (1, 1252)	0.1149458000092495
  (1, 3065)	0.2354898541892898
  (1, 345)	0.23411311241086843
  (1, 1786)	0.1528197188369887
  (1, 2765)	0.04957577042271937
  (1, 2777)	0.03715528248421348
  (1, 553)	0.04474627024092432
  (1, 1664)	0.2778634284016085
  (1, 3985)	0.25247426087747765
  :	:
  (12020, 2814)	0.11954687400731291
  (12021, 3014)	0.3303092754497183
  (12021, 3548)	0.2848002270917831
  (12021, 3483)	0.27506679660590744
  (12021, 4167)	0.25622178313404537
  (12021, 242)	0.2814527076726

In [108]:
tf_vec = tf_vec.toarray()

In [123]:
# vectorize 후 keyword_list_literal 컬럼에 저장 (위에서부터 실행시 활용)
# train_key_sum["keyword_list_literal"] = train_key_sum['morphs_list'].progress_apply(lambda x: (" ").join(x))
# tf_vec = tfidf_vectorizer.fit_transform(train_key_sum['keyword_list_literal'])
# print(tf_vec)



  0%|          | 0/12022 [00:00<?, ?it/s]

 33%|███▎      | 3955/12022 [00:00<00:00, 37061.92it/s]

100%|██████████| 12022/12022 [00:00<00:00, 53254.90it/s]


  (0, 2596)	0.08892226356721587
  (0, 2493)	0.7681460744707492
  (0, 285)	0.4326307307899682
  (0, 3154)	0.46354621138932706
  (1, 2530)	0.1108507957519865
  (1, 2535)	0.10660355469600799
  (1, 717)	0.11489996573920543
  (1, 2405)	0.2888101083568638
  (1, 220)	0.45439651793448177
  (1, 3369)	0.06282610806441084
  (1, 1351)	0.09543601466677015
  (1, 2450)	0.07668297764741876
  (1, 1888)	0.12762453906603943
  (1, 2999)	0.0641434745525184
  (1, 2956)	0.20192143601667717
  (1, 4065)	0.20260131208375642
  (1, 1252)	0.1149458000092495
  (1, 3065)	0.2354898541892898
  (1, 345)	0.23411311241086843
  (1, 1786)	0.1528197188369887
  (1, 2765)	0.04957577042271937
  (1, 2777)	0.03715528248421348
  (1, 553)	0.04474627024092432
  (1, 1664)	0.2778634284016085
  (1, 3985)	0.25247426087747765
  :	:
  (12020, 2814)	0.11954687400731291
  (12021, 3014)	0.3303092754497183
  (12021, 3548)	0.2848002270917831
  (12021, 3483)	0.27506679660590744
  (12021, 4167)	0.25622178313404537
  (12021, 242)	0.2814527076726

## cosine 유사도 구하기

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

reader_keyword_sim = cosine_similarity(tf_vec, tf_vec)
print(reader_keyword_sim.shape)
print(reader_keyword_sim[:2])

(12022, 12022)
[[1.         0.         0.         ... 0.         0.         0.29783083]
 [0.         1.         0.02913224 ... 0.00603721 0.00929597 0.01749004]]


In [98]:
reader_keyword_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.29783083],
       [0.        , 1.        , 0.02913224, ..., 0.00603721, 0.00929597,
        0.01749004],
       [0.        , 0.02913224, 1.        , ..., 0.        , 0.01691538,
        0.00757019],
       ...,
       [0.        , 0.00603721, 0.        , ..., 1.        , 0.04325531,
        0.01750022],
       [0.        , 0.00929597, 0.01691538, ..., 0.04325531, 1.        ,
        0.03357306],
       [0.29783083, 0.01749004, 0.00757019, ..., 0.01750022, 0.03357306,
        1.        ]])

In [99]:
keyword_sim_sorted_ind = reader_keyword_sim.argsort()[:,::-1]
print(keyword_sim_sorted_ind[:1])

[[   0 7857 6058 ... 6262 6268 6010]]


In [100]:
# 유사한 reader 찾아주는 함수
def find_sim_article(df, sorted_ind, readers_name, top_n=10):
    reader_list = df[df['readers_id']==readers_name]
    
    article_index = reader_list.index.values
    similar_indexes = sorted_ind[article_index, :(top_n)]
    
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    

    return df.iloc[similar_indexes]

In [101]:
train_key_sum

,readers_id,keyword_list,morphs_list,keyword_list_literal
0,#0013c35c789f0936654ba06389ea75cb,"['육아', '결혼', '시어머니', '육아', '결혼', '시어머니', '결혼',...","육아, 결혼, 시어머니, 육아, 결혼, 시어머니, 결혼, 육아, 시어머니, 시어머니...","육아, 결혼, 시어머니, 육아, 결혼, 시어머니, 결혼, 육아, 시어머니, 시어머니..."
1,#00222a855547061a7fd00e404cb96148,"['음식', '일본여행', '추천', '교실영어', 'English', '문장', ...","음식, 일본, 여행, 추천, 교실, 영어, English, 문장, 이력서, 면접, ...","음식, 일본, 여행, 추천, 교실, 영어, English, 문장, 이력서, 면접, ..."
2,#002cedb579c73f021b86f8b12d8e35d8,"['호텔', '서울홍대', '여행', '호텔', '서울홍대', '여행', '세계정상...","호텔, 서울, 홍, 대, 여행, 호텔, 서울, 홍, 대, 여행, 세계, 정상, 정상...","호텔, 서울, 홍, 대, 여행, 호텔, 서울, 홍, 대, 여행, 세계, 정상, 정상..."
3,#002f5cac6b2c802ec323d3d3424a71cf,"['연애', '감정', '사랑', '연애', '감정', '사랑', '남편', '신혼...","연애, 감정, 사랑, 연애, 감정, 사랑, 남편, 신혼, 일기, 신혼, 여행, 광,...","연애, 감정, 사랑, 연애, 감정, 사랑, 남편, 신혼, 일기, 신혼, 여행, 광,..."
4,#00339d09d3f964d88ae3e57a81803539,"['베이킹소다', '청소', '봄맞이', '결혼', '부부', '사랑', '방콕',...","베이, 킹, 소다, 청소, 봄맞이, 결혼, 부부, 사랑, 방, 콕, 태국, 야시장,...","베이, 킹, 소다, 청소, 봄맞이, 결혼, 부부, 사랑, 방, 콕, 태국, 야시장,..."
...,...,...,...,...
12017,#fff10b2ae2f6e5a121a0adea7062898d,"['건강검진', '갑상선암', '건강', '건강검진', '갑상선암', '건강', '...","건강, 검진, 갑상선, 암, 건강, 건강, 검진, 갑상선, 암, 건강, 건강, 검진...","건강, 검진, 갑상선, 암, 건강, 건강, 검진, 갑상선, 암, 건강, 건강, 검진..."
12018,#fff243d06ee500cd9af1fdfe7886bac7,"['아침생각', '생각', '완벽', '아침생각', '생각', '완벽', '아침생각...","아침, 생각, 생각, 완벽, 아침, 생각, 생각, 완벽, 아침, 생각, 생각, 완벽...","아침, 생각, 생각, 완벽, 아침, 생각, 생각, 완벽, 아침, 생각, 생각, 완벽..."
12019,#fff892683009fe46182af61b859d2413,"['스트레칭', '운동', '척추', '스트레칭', '운동', '척추', '고양이'...","스트레칭, 운동, 척추, 스트레칭, 운동, 척추, 고양이, 길, 고양이, 고양이, ...","스트레칭, 운동, 척추, 스트레칭, 운동, 척추, 고양이, 길, 고양이, 고양이, ..."
12020,#fffca5f83f8b7145b3102aec2e0934dc,"['여행', '국내여행', '골든리트리버', '여행', '국내여행', '골든리트리버...","여행, 국내, 여행, 골든, 리, 트리, 벌, 여행, 국내, 여행, 골든, 리, 트...","여행, 국내, 여행, 골든, 리, 트리, 벌, 여행, 국내, 여행, 골든, 리, 트..."


In [102]:
train_key_sum[train_key_sum['readers_id']=='#002cedb579c73f021b86f8b12d8e35d8']['keyword_list']

2    ['호텔', '서울홍대', '여행', '호텔', '서울홍대', '여행', '세계정상...
Name: keyword_list, dtype: object

In [105]:
similar_article[['readers_id', 'keyword_list']].iloc[0][1]

"['호텔', '서울홍대', '여행', '호텔', '서울홍대', '여행', '세계정상', '정상', '맥주', '세계정상', '정상', '맥주', '브런치', '시사회', '브런치무비패스', '브런치', '시사회', '브런치무비패스', '거짓말', '연극', '영화', '거짓말', '연극', '영화', '거짓말', '연극', '영화', '거짓말', '연극', '영화', '놀이터', '놀이', '어린이', '놀이터', '놀이', '어린이', '영국', '미국', '배송', '영국', '미국', '배송', '영국', '미국', '배송', '영국', '미국', '배송', '곡물음료', '아침음료', '음료', '곡물음료', '아침음료', '음료', '곡물음료', '아침음료', '음료', '곡물음료', '아침음료', '음료', '임신', '엄마', '아이', '워터젤리', '젤리', '편의점', '워터젤리', '젤리', '편의점', '남녀', '신부', '신랑', '남녀', '신부', '신랑', '호텔', '중국', '럭셔리', '호텔', '중국', '럭셔리']"

In [106]:
similar_article[['readers_id', 'keyword_list']].iloc[1][1]

"['인터뷰', '작가', '아나운서', '인터뷰', '작가', '아나운서', '인터뷰', '작가', '아나운서', '인터뷰', '작가', '아나운서', '문화기획', '기획', '문화기획', '기획', '문화기획', '기획', '문화기획', '기획', '하루키', '일상', '에세이', '직장생활', '에세이', '드라마', '직장생활', '에세이', '드라마', '직장생활', '에세이', '드라마', '직장생활', '에세이', '드라마', '독서', '개념', '에세이', '독서', '개념', '에세이', '직장인', '퇴사', '상사', '직장인', '퇴사', '상사', '홍조일기', '고양이', '웹툰', '세계정상', '정상', '맥주', '세계정상', '정상', '맥주', '세계정상', '정상', '맥주', '세계정상', '정상', '맥주', '브런치', '시사회', '브런치무비패스', '브런치', '시사회', '브런치무비패스', '브런치', '시사회', '브런치무비패스', '브런치', '시사회', '브런치무비패스', '놀이터', '놀이', '어린이', '곡물음료', '아침음료', '음료', '곡물음료', '아침음료', '음료', '곡물음료', '아침음료', '음료', '곡물음료', '아침음료', '음료', '역사', '여성', '프랑스', '크라우드펀딩', '독립출판', '텀블벅', '크라우드펀딩', '독립출판', '텀블벅', '크라우드펀딩', '독립출판', '텀블벅', '워터젤리', '젤리', '편의점', '워터젤리', '젤리', '편의점', '워터젤리', '젤리', '편의점', '워터젤리', '젤리', '편의점', '나무', '사람', '변신', '나무', '사람', '변신', '나무', '사람', '변신', '나무', '사람', '변신', '나무', '사람', '변신', '일러스트', '에세이', '캘리그라피', '영화', '글쓰기', '영화리뷰', '영화', '글쓰기', '영화리뷰', '영화', '글쓰기', '영화리뷰', '영화', '글쓰기

In [104]:
similar_article = find_sim_article(train_key_sum, keyword_sim_sorted_ind, '#002cedb579c73f021b86f8b12d8e35d8', 10)
similar_article[['readers_id', 'keyword_list']]

[[   2 5243 4412 4767 5871 1093 3584 8296 7599 1713]]


,readers_id,keyword_list
2,#002cedb579c73f021b86f8b12d8e35d8,"['호텔', '서울홍대', '여행', '호텔', '서울홍대', '여행', '세계정상..."
5243,#703bffcb38d1a5c285c4c521c5b1d438,"['인터뷰', '작가', '아나운서', '인터뷰', '작가', '아나운서', '인터..."
4412,#5e7db50b7f42dad9e3be72bc3e8cae4c,"['일본여행', '도쿄여행', '손수건', '명절', '친정엄마', '추석', '쌀..."
4767,#65ce6aa8ff6340e78f31137f462b7c34,"['연애', '감정', '사랑', '호감', '소개팅', '첫만남', '남편', '..."
5871,#7c6d0af39aed018d058f98ebee876da2,"['육아', '육아일기', '아빠', '일본', '손수건', '여행', '일본', ..."
1093,#17b6957bfd45aeabe3a38757edff5914,"['고양이', '길고양이', '반려동물', '고양이', '길고양이', '반려동물',..."
3584,#4c9b355fb7cf7b6b73269d3d93008eb7,"['투자', '버핏', '크래프트', '울산', '집값', '창원', '카피', '..."
8296,#b0abe77bf4d91013f0969c8685db0c02,"['독서', '소설', '소설', '에세이', '무라카미하루키', '소설', '에세..."
7599,#a21b9948d1e7586e9cc6e001c2d7cc66,"['캐나다', '옐로나이프', '오로라', '책소개', '도서', '에세이', '출..."
1713,#24db8364bfa65b806d06da93fb2ddf30,"['스트레칭', '운동', '척추', '스트레칭', '운동', '척추', '여행',..."


# 작가-작가 유사도 구하기

## 작가별 키워드 말뭉치 구하기

In [6]:
# 파일 read하기
author_key_sum = meta.groupby('author_id')['keyword_list'].agg('sum')
author_key_sum

author_id
@002                                 [계란, 살충제, 피해, 귀촌, 농촌, 청년농업인]
@002jesus                                           [후원, 기부, 비영리]
@002paper       [제주, 서울생활, 서울, 한마디, 감동, 북카페, 바램, 일상에세이, 긍정, 여행...
@008hood        [김춘수, 신영복, 영화, 광주, 한강, 소년, 일본, 일본사, 외교관, 가난, 도...
@00books        [최진영, 죽음, 소설, 트랜스젠더, 인권, 드라마, 프로야구, 야구, 백인천프로젝...
                                      ...                        
@zzumit         [정리, 정리여행, 여행, 스콘, 커피, 브라이튼, 커피, 음악, 추모, 커피, 다...
@zzyoun         [영화, 로건럭키, 브런치무비패스, 영화, 대부, 리뷰, 쿠엔틴타란티노, 영화평론,...
@zzz1004jang    [쓰레기통, 사랑, 이유, 해물칼국수, 허기, 거짓말, 역대최고점, 실내온도, 채점...
@zzzaam         [UX, 디자인, UX, 알렉사, design, UX, 작가, 글쓰기, 인공지능, ...
@zzzwhite       [법정스님, 세월, 드로잉, 드로잉, 그림일기, 감성사진, 감성그림, 희망, 엄마,...
Name: keyword_list, Length: 19065, dtype: object

In [7]:
author_key_sum = author_key_sum.reset_index()
author_key_sum

,author_id,keyword_list
0,@002,"[계란, 살충제, 피해, 귀촌, 농촌, 청년농업인]"
1,@002jesus,"[후원, 기부, 비영리]"
2,@002paper,"[제주, 서울생활, 서울, 한마디, 감동, 북카페, 바램, 일상에세이, 긍정, 여행..."
3,@008hood,"[김춘수, 신영복, 영화, 광주, 한강, 소년, 일본, 일본사, 외교관, 가난, 도..."
4,@00books,"[최진영, 죽음, 소설, 트랜스젠더, 인권, 드라마, 프로야구, 야구, 백인천프로젝..."
...,...,...
19060,@zzumit,"[정리, 정리여행, 여행, 스콘, 커피, 브라이튼, 커피, 음악, 추모, 커피, 다..."
19061,@zzyoun,"[영화, 로건럭키, 브런치무비패스, 영화, 대부, 리뷰, 쿠엔틴타란티노, 영화평론,..."
19062,@zzz1004jang,"[쓰레기통, 사랑, 이유, 해물칼국수, 허기, 거짓말, 역대최고점, 실내온도, 채점..."
19063,@zzzaam,"[UX, 디자인, UX, 알렉사, design, UX, 작가, 글쓰기, 인공지능, ..."


In [8]:
len(author_key_sum)

19065

In [11]:
# 작가별 주로 사용한 글의 키워드
author_key_sum = author_key_sum[author_key_sum['keyword_list'].apply(lambda x: len(x))!=0]
author_key_sum

,author_id,keyword_list
0,@002,"[계란, 살충제, 피해, 귀촌, 농촌, 청년농업인]"
1,@002jesus,"[후원, 기부, 비영리]"
2,@002paper,"[제주, 서울생활, 서울, 한마디, 감동, 북카페, 바램, 일상에세이, 긍정, 여행..."
3,@008hood,"[김춘수, 신영복, 영화, 광주, 한강, 소년, 일본, 일본사, 외교관, 가난, 도..."
4,@00books,"[최진영, 죽음, 소설, 트랜스젠더, 인권, 드라마, 프로야구, 야구, 백인천프로젝..."
...,...,...
19060,@zzumit,"[정리, 정리여행, 여행, 스콘, 커피, 브라이튼, 커피, 음악, 추모, 커피, 다..."
19061,@zzyoun,"[영화, 로건럭키, 브런치무비패스, 영화, 대부, 리뷰, 쿠엔틴타란티노, 영화평론,..."
19062,@zzz1004jang,"[쓰레기통, 사랑, 이유, 해물칼국수, 허기, 거짓말, 역대최고점, 실내온도, 채점..."
19063,@zzzaam,"[UX, 디자인, UX, 알렉사, design, UX, 작가, 글쓰기, 인공지능, ..."


## 키워드 형태소 분석

In [15]:
tqdm.pandas()
author_key_sum['morphs_list'] = author_key_sum['keyword_list'].progress_apply(lambda x: morphs_analysis(x))


  3%|▎         | 602/17860 [05:12<1:50:15,  2.61it/s]


  7%|▋         | 1198/17860 [10:54<1:23:13,  3.34it/s]


 10%|█         | 1793/17860 [17:41<2:02:09,  2.19it/s]


 13%|█▎        | 2401/17860 [22:53<57:16,  4.50it/s]


 17%|█▋        | 2982/17860 [27:13<2:20:47,  1.76it/s]


...

 87%|████████▋ | 15549/17860 [2:12:24<11:38,  3.31it/s]


 90%|█████████ | 16141/17860 [2:18:40<20:45,  1.38it/s]


 94%|█████████▎| 16709/17860 [2:23:58<13:43,  1.40it/s]


 97%|█████████▋| 17353/17860 [2:28:18<02:44,  3.08it/s]


100%|██████████| 17860/17860 [2:33:30<00:00,  1.94it/s]


In [16]:
author_key_sum.to_csv('all_author_morphs_duplicates.csv', encoding='utf-8')

In [ ]:
## 형태소 분석 후 tf-idf vectorizer 구성

## Tfidf vectorizer 구성

In [107]:
author_key_sum

,author_id,keyword_list,morphs_list,keyword_list_literal
0,@002,"['계란', '살충제', '피해', '귀촌', '농촌', '청년농업인']","계란, 살충제, 피해, 귀, 촌, 농촌, 청년, 농업, 이, ㄴ","계란, 살충제, 피해, 귀, 촌, 농촌, 청년, 농업, 이, ㄴ"
1,@002jesus,"['후원', '기부', '비영리']","후원, 기부, 비영리","후원, 기부, 비영리"
2,@002paper,"['제주', '서울생활', '서울', '한마디', '감동', '북카페', '바램',...","제주, 서울, 생활, 서울, 한마디, 감동, 북, 카페, 바램, 일상, 에, 세, ...","제주, 서울, 생활, 서울, 한마디, 감동, 북, 카페, 바램, 일상, 에, 세, ..."
3,@008hood,"['김춘수', '신영복', '영화', '광주', '한강', '소년', '일본', '...","김춘수, 신, 영복, 영화, 광주, 한강, 소년, 일본, 일본, 사, 외교관, 가난...","김춘수, 신, 영복, 영화, 광주, 한강, 소년, 일본, 일본, 사, 외교관, 가난..."
4,@00books,"['최진영', '죽음', '소설', '트랜스젠더', '인권', '드라마', '프로야...","최, 진영, 죽음, 소설, 트랜스, 젠더, 인권, 드라마, 프로, 야구, 야구, 백...","최, 진영, 죽음, 소설, 트랜스, 젠더, 인권, 드라마, 프로, 야구, 야구, 백..."
...,...,...,...,...
17855,@zzumit,"['정리', '정리여행', '여행', '스콘', '커피', '브라이튼', '커피',...","정리, 정리, 여행, 여행, 스콘, 커피, 브라, 이, 트, ㄴ, 커피, 음악, 추...","정리, 정리, 여행, 여행, 스콘, 커피, 브라, 이, 트, ㄴ, 커피, 음악, 추..."
17856,@zzyoun,"['영화', '로건럭키', '브런치무비패스', '영화', '대부', '리뷰', '쿠...","영화, 로건럭키, 브런, 하, 지, 무비, 패스, 영화, 대부, 리뷰, 쿠, 엔, ...","영화, 로건럭키, 브런, 하, 지, 무비, 패스, 영화, 대부, 리뷰, 쿠, 엔, ..."
17857,@zzz1004jang,"['쓰레기통', '사랑', '이유', '해물칼국수', '허기', '거짓말', '역대...","쓰레기통, 사랑, 이유, 해물, 칼국수, 허기, 거짓말, 역대, 최고점, 실내온도,...","쓰레기통, 사랑, 이유, 해물, 칼국수, 허기, 거짓말, 역대, 최고점, 실내온도,..."
17858,@zzzaam,"['UX', '디자인', 'UX', '알렉사', 'design', 'UX', '작가...","UX, 디자인, UX, 알, ㄹ, 렉, 사, design, UX, 작가, 글쓰기, ...","UX, 디자인, UX, 알, ㄹ, 렉, 사, design, UX, 작가, 글쓰기, ..."


In [109]:
# 저장파일 불러오기
author_key_sum = pd.read_csv('all_author_morphs_duplicates.csv', encoding='utf-8', index_col=[0]).reset_index(drop=True)

In [110]:
author_key_sum['morphs_list'] = author_key_sum['morphs_list'].apply(lambda x: str(x).replace('[', '').replace(']','').replace("'", ''))

In [111]:
author_key_sum = author_key_sum[author_key_sum['keyword_list'].apply(lambda x: len(x))!=0]
author_key_sum

,author_id,keyword_list,morphs_list
0,@002,"['계란', '살충제', '피해', '귀촌', '농촌', '청년농업인']","계란, 살충제, 피해, 귀, 촌, 농촌, 청년, 농업, 이, ㄴ"
1,@002jesus,"['후원', '기부', '비영리']","후원, 기부, 비영리"
2,@002paper,"['제주', '서울생활', '서울', '한마디', '감동', '북카페', '바램',...","제주, 서울, 생활, 서울, 한마디, 감동, 북, 카페, 바램, 일상, 에, 세, ..."
3,@008hood,"['김춘수', '신영복', '영화', '광주', '한강', '소년', '일본', '...","김춘수, 신, 영복, 영화, 광주, 한강, 소년, 일본, 일본, 사, 외교관, 가난..."
4,@00books,"['최진영', '죽음', '소설', '트랜스젠더', '인권', '드라마', '프로야...","최, 진영, 죽음, 소설, 트랜스, 젠더, 인권, 드라마, 프로, 야구, 야구, 백..."
...,...,...,...
17855,@zzumit,"['정리', '정리여행', '여행', '스콘', '커피', '브라이튼', '커피',...","정리, 정리, 여행, 여행, 스콘, 커피, 브라, 이, 트, ㄴ, 커피, 음악, 추..."
17856,@zzyoun,"['영화', '로건럭키', '브런치무비패스', '영화', '대부', '리뷰', '쿠...","영화, 로건럭키, 브런, 하, 지, 무비, 패스, 영화, 대부, 리뷰, 쿠, 엔, ..."
17857,@zzz1004jang,"['쓰레기통', '사랑', '이유', '해물칼국수', '허기', '거짓말', '역대...","쓰레기통, 사랑, 이유, 해물, 칼국수, 허기, 거짓말, 역대, 최고점, 실내온도,..."
17858,@zzzaam,"['UX', '디자인', 'UX', '알렉사', 'design', 'UX', '작가...","UX, 디자인, UX, 알, ㄹ, 렉, 사, design, UX, 작가, 글쓰기, ..."


In [73]:
tfidf_vectorizer = TfidfVectorizer()

In [112]:
# 저장파일 불러왔을 때 활용
tqdm.pandas()
# author_key_sum["keyword_list_literal"] = author_key_sum['morphs_list'].progress_apply(lambda x: ("").join(x))
author_tf_vec = tfidf_vectorizer.transform(author_key_sum['morphs_list'])
print(author_tf_vec)

C:\anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


  (0, 4456)	0.5156054120425891
  (0, 3855)	0.30889278710473067
  (0, 809)	0.5156054120425891
  (0, 807)	0.3655074536009107
  (0, 311)	0.48917447355890664
  (1, 618)	1.0
  (2, 4533)	0.11366228267577097
  (2, 4518)	0.27546564922933514
  (2, 3991)	0.1045894238259867
  (2, 3739)	0.17765913951672296
  (2, 3588)	0.4682164453678493
  (2, 3542)	0.2594617742668298
  (2, 3328)	0.0757750055066067
  (2, 3293)	0.08525370340490075
  (2, 3225)	0.14649089347225552
  (2, 2950)	0.26769435379348994
  (2, 2814)	0.10789840652627257
  (2, 2801)	0.4365454080445548
  (2, 2113)	0.3054810614646709
  (2, 2097)	0.06083155975923185
  (2, 1943)	0.12030286147183752
  (2, 1845)	0.21505567015820123
  (2, 590)	0.2107268232226146
  (2, 185)	0.26769435379348994
  (3, 4507)	0.3126562935514954
  :	:
  (17857, 249)	0.3223948662154159
  (17858, 3509)	0.09843186230800983
  (17858, 3414)	0.07293871336751982
  (17858, 3298)	0.18595238011121248
  (17858, 3273)	0.28225047257019753
  (17858, 3172)	0.188520535083529
  (17858, 1963)

In [113]:
author_tf_vec = author_tf_vec.toarray()

In [28]:
# vectorize 후 keyword_list_literal 컬럼에 저장 (위에서부터 실행시 활용)
# train_key_sum["keyword_list_literal"] = train_key_sum['morphs_list'].progress_apply(lambda x: (" ").join(x))
# tf_vec = tfidf_vectorizer.fit_transform(train_key_sum['keyword_list_literal'])
# print(tf_vec)

## cosine 유사도 구하기

In [29]:
from sklearn.metrics.pairwise import cosine_similarity

author_keyword_sim = cosine_similarity(author_tf_vec, author_tf_vec)
print(author_keyword_sim.shape)
print(author_keyword_sim[:2])

(3099, 3099)
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [30]:
author_keyword_sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [31]:
keyword_sim_sorted_ind = author_keyword_sim.argsort()[:,::-1]
print(keyword_sim_sorted_ind[:1])

[[   0 1064 1038 ... 2061 2060 1549]]


In [32]:
# 유사한 reader 찾아주는 함수
def find_sim_author(df, sorted_ind, authors_name, top_n=10):
    reader_list = df[df['author_id']==authors_name]
    
    article_index = reader_list.index.values
    similar_indexes = sorted_ind[article_index, :(top_n)]
    
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    

    return df.iloc[similar_indexes]

In [33]:
author_key_sum

,author_id,keyword_list,morphs_list,keyword_list_literal
1,@01026856762,"[부모교육, 학교폭력, 가정, 부모교육, 학교폭력, 가정, 부모교육, 학교폭력, 가...","[부모, 교육, 학교, 폭력, 가정, 부모, 교육, 학교, 폭력, 가정, 부모, 교...",부모교육학교폭력가정부모교육학교폭력가정부모교육학교폭력가정부모교육학교폭력가정부모교육학교...
2,@01038273527,"[요리, 요리법, 저녁식사, 비전, 자기관리, 꿈, 비전, 자기관리, 꿈, 비전, ...","[요리, 요리법, 저녁, 식사, 비전, 자기관리, 꿈, 비전, 자기관리, 꿈, 비전...",요리요리법저녁식사비전자기관리꿈비전자기관리꿈비전자기관리꿈비전자기관리꿈자존감취업희망요리...
3,@01archive,"[영화, 사랑, 영화, 사랑, 영화, 계곡, 영화, 계곡, 영화, 계곡, 영화, 계...","[영화, 사랑, 영화, 사랑, 영화, 계곡, 영화, 계곡, 영화, 계곡, 영화, 계...",영화사랑영화사랑영화계곡영화계곡영화계곡영화계곡영화사랑영화사랑영화계곡영화사랑영화계곡영화...
4,@02jsh0314,"[영어유치원, 영어, 유치원, 영어유치원, 영어, 유치원, 영어유치원, 영어, 유치...","[영어, 유치원, 영어, 유치원, 영어, 유치원, 영어, 유치원, 영어, 유치원, ...",영어유치원영어유치원영어유치원영어유치원영어유치원영어유치원영어유치원영어유치원영어유치원영...
5,@0327,"[커피, 여행, 카페, 커피, 여행, 카페, 일본, 일본어, 여행, 일본, 일본어,...","[커피, 여행, 카페, 커피, 여행, 카페, 일본, 일본어, 여행, 일본, 일본어,...",커피여행카페커피여행카페일본일본어여행일본일본어여행커피여행카페커피여행카페커피여행카페커피...
...,...,...,...,...
3167,@zungowl,"[아빠, 그림, 가족, 아빠, 그림, 가족, 아빠, 그림, 가족]","[아빠, 그림, 가족, 아빠, 그림, 가족, 아빠, 그림, 가족]",아빠그림가족아빠그림가족아빠그림가족
3168,@zyeun,"[일상, 에세이, 생각, 일상, 에세이, 생각, 일상, 에세이, 생각, 일상, 에세...","[일상, 에세이, 생각, 일상, 에세이, 생각, 일상, 에세이, 생각, 일상, 에세...",일상에세이생각일상에세이생각일상에세이생각일상에세이생각일상에세이생각이별에세이연애이별에세...
3169,@zzing-gu,"[공감에세이, 글쓰기, 시, 공감에세이, 글쓰기, 시, 공감에세이, 글쓰기, 시, ...","[공감, 에, 세, 이, 글쓰기, 시, 공감, 에, 세, 이, 글쓰기, 시, 공감,...",공감에세이글쓰기시공감에세이글쓰기시공감에세이글쓰기시글쓰기쓸쓸시공감에세이글쓰기시공감에세...
3170,@zzintta,"[웹툰, 웹소설, 좀비, 웹툰, 웹소설, 좀비, 웹툰, 웹소설, 좀비]","[웹, 툰, 웹, 소설, 좀비, 웹, 툰, 웹, 소설, 좀비, 웹, 툰, 웹, 소설...",웹툰웹소설좀비웹툰웹소설좀비웹툰웹소설좀비


In [34]:
author_key_sum[author_key_sum['author_id']=='@13july']['keyword_list']

14    [미니멀라이프, 라이프스타일, 일러스트레이션, 중고거래, 일러스트레이션, 미니멀리스...
Name: keyword_list, dtype: object

In [38]:
similar_article[['author_id', 'keyword_list']].iloc[0][1]

['복숭아',
 '여름바다',
 '사랑',
 '영화',
 '심야영화',
 '사랑',
 '영화',
 '심야영화',
 '사랑',
 '사랑',
 '생각',
 '복숭아',
 '여름바다',
 '사랑',
 '복숭아',
 '여름바다',
 '사랑',
 '복숭아',
 '여름바다',
 '사랑',
 '사랑',
 '생각',
 '영화',
 '어벤져스',
 '히어로물',
 '사랑',
 '상실',
 '고독',
 '영화',
 '심야영화',
 '사랑',
 '사랑',
 '상실',
 '고독',
 '사랑',
 '상실',
 '고독',
 '자신',
 '이해',
 '심리',
 '자신',
 '이해',
 '심리',
 '박하사탕',
 '순간',
 '영화',
 '박하사탕',
 '순간',
 '영화',
 '박하사탕',
 '순간',
 '영화',
 '박하사탕',
 '순간',
 '영화',
 '박하사탕',
 '순간',
 '영화',
 '박하사탕',
 '순간',
 '영화',
 '영화',
 '어벤져스',
 '히어로물',
 '우주',
 '순간',
 '명왕성',
 '마음',
 '한강',
 '사랑',
 '마음',
 '한강',
 '사랑',
 '자신',
 '이해',
 '심리',
 '생각',
 '연애',
 '생각',
 '연애',
 '자신',
 '싸움',
 '자아',
 '라이너스의담요',
 '겨울왕국',
 '사용',
 '자신',
 '이해',
 '심리',
 '자신',
 '이해',
 '심리',
 '자신',
 '이해',
 '심리',
 '자신',
 '이해',
 '심리',
 '루소',
 '교육',
 '무기력',
 '자신',
 '이해',
 '심리',
 '자신',
 '이해',
 '심리',
 '자신',
 '이해',
 '심리',
 '자신',
 '이해',
 '심리',
 '루소',
 '교육',
 '무기력',
 '자신',
 '이해',
 '심리',
 '자신',
 '이해',
 '심리',
 '자신',
 '이해',
 '심리',
 '자신',
 '이해',
 '심리',
 '자신',
 '이해',
 '심리',
 '자신',
 '이해',
 '심리',
 '자신',
 '이해',


In [39]:
similar_article[['author_id', 'keyword_list']].iloc[1][1]

['캘리그라피',
 '시',
 '글쓰기',
 '캘리그라피',
 '시',
 '글쓰기',
 '그림',
 '캘리그라피',
 '봄',
 '그림',
 '캘리그라피',
 '봄',
 '그림',
 '캘리그라피',
 '봄',
 '그림',
 '캘리그라피',
 '봄',
 '캘리그라피',
 '시',
 '글쓰기',
 '캘리그라피',
 '시',
 '글쓰기',
 '그림',
 '캘리그라피',
 '봄',
 '캘리그라피',
 '시',
 '글쓰기',
 '캘리그라피',
 '시',
 '글쓰기',
 '캘리그라피',
 '시',
 '글쓰기',
 '캘리그라피',
 '시',
 '글쓰기',
 '캘리그라피',
 '시',
 '글쓰기',
 '캘리그라피',
 '시',
 '글쓰기',
 '그림',
 '캘리그라피',
 '봄',
 '캘리그라피',
 '시',
 '글쓰기',
 '캘리그라피',
 '시',
 '글쓰기',
 '캘리그라피',
 '시',
 '글쓰기',
 '캘리그라피',
 '시',
 '글쓰기',
 '그림',
 '캘리그라피',
 '봄',
 '그림',
 '캘리그라피',
 '봄',
 '캘리그라피',
 '시',
 '글쓰기',
 '캘리그라피',
 '시',
 '글쓰기',
 '캘리그라피',
 '시',
 '글쓰기',
 '캘리그라피',
 '시',
 '글쓰기',
 '그림',
 '캘리그라피',
 '봄',
 '그림',
 '캘리그라피',
 '봄',
 '캘리그라피',
 '시',
 '글쓰기',
 '캘리그라피',
 '시',
 '글쓰기',
 '캘리그라피',
 '시',
 '글쓰기',
 '캘리그라피',
 '시',
 '글쓰기',
 '그림',
 '캘리그라피',
 '봄',
 '그림',
 '캘리그라피',
 '봄',
 '캘리그라피',
 '시',
 '글쓰기',
 '캘리그라피',
 '시',
 '글쓰기',
 '캘리그라피',
 '시',
 '글쓰기',
 '그림',
 '캘리그라피',
 '봄',
 '그림',
 '캘리그라피',
 '봄',
 '캘리그라피',
 '시',
 '글쓰기',
 '캘리그라피',
 '시',
 '글쓰기',
 '캘리그라피',
 '시',
 '글쓰기',
 '캘리그라

In [37]:
similar_article = find_sim_author(author_key_sum, keyword_sim_sorted_ind, '@13july', 10)
similar_article[['author_id', 'keyword_list']]

[[  14 3098 1029 1038 1037 1036 1035 1034 1033 1032]]


,author_id,keyword_list
15,@1khhan,"[복숭아, 여름바다, 사랑, 영화, 심야영화, 사랑, 영화, 심야영화, 사랑, 사랑..."
3171,@zzolmarkb6sm,"[캘리그라피, 시, 글쓰기, 캘리그라피, 시, 글쓰기, 그림, 캘리그라피, 봄, 그..."
1054,@hasungtae,"[KBS, 저널리즘토크쇼, 조선일보, KBS, 저널리즘, 조선일보, 박근혜, 손석희..."
1064,@hazzibo,"[수필, 오늘, 내일, 수필, 오늘, 내일, 단상, 생각, 나, 단상, 생각, 나,..."
1063,@hazy-day,"[디저트, 롤케이크, 디저트, 롤케이크, 디저트, 롤케이크, 디저트, 롤케이크, 디..."
1062,@hayunoh19,"[독서, 소설, 책리뷰, 영화, 빈센트반고흐, 영화리뷰, 영화, 빈센트반고흐, 영화..."
1061,@haydenmun,"[뉴스, 루머, 언론, 뉴스, 루머, 언론, 뉴스, 루머, 언론, 뉴스, 루머, 언론]"
1059,@hawann,"[그림에세이, 리틀포레스트, 영화, 그림에세이, 리틀포레스트, 영화, 그림에세이, ..."
1058,@hawaiikaikitten,"[크루즈, 해외여행, 사진에세이, 크루즈, 해외여행, 사진에세이, 크루즈, 해외여행..."
1057,@havein,"[크리에이터, 전시, 디자인, 크리에이터, 전시, 디자인, 크리에이터, 전시, 디자..."


----

> 아래부터는 다시 시도해야함

## 작가별 말뭉치 tf-idf로 벡터화하기

# 독자-작가 유사도 구하기

- 독자 키워드 리스트와 작가 키워드 리스트를 넣고 cosine유사도를 구함
- 독자 id를 넣었을 때 유사 작가 id가 나오도록 함수 설정

## 앞에서 구한 독자 키워드, 작가 키워드로 cosine유사도 구하기
- 키워드는 벡터변환 후 toarray()까지 해서 유사도를 구해야 함
- 독자 키워드 리스트 말뭉치 vector화 변수: tf_vec
- 작가 키워드 리스트 말뭉치 vector화 변수: authoor_tf_vec

In [76]:
tf_vec.shape

(12022, 4762)

In [77]:
author_tf_vec.shape

(17860, 25795)

In [114]:
# 독자-작가 코사인 유사도 구하기
from sklearn.metrics.pairwise import cosine_similarity

reader_author_keyword_sim = cosine_similarity(tf_vec, author_tf_vec)
print(reader_author_keyword_sim.shape)
print(reader_author_keyword_sim[:2])

(12022, 17860)
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.00385413 ... 0.         0.         0.02043549]]


In [115]:
# 코사인 유사도 결과물
reader_author_keyword_sim

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.00385413, ..., 0.        , 0.        ,
        0.02043549],
       [0.        , 0.        , 0.02571891, ..., 0.08258241, 0.        ,
        0.00177589],
       ...,
       [0.        , 0.        , 0.00546589, ..., 0.04595578, 0.00622205,
        0.12047844],
       [0.        , 0.        , 0.0149489 , ..., 0.        , 0.01166791,
        0.06529667],
       [0.        , 0.        , 0.01855172, ..., 0.        , 0.        ,
        0.        ]])

In [116]:
# 작가인덱스
author_key_sum['author_id']

0                @002
1           @002jesus
2           @002paper
3            @008hood
4            @00books
             ...     
17855         @zzumit
17856         @zzyoun
17857    @zzz1004jang
17858         @zzzaam
17859       @zzzwhite
Name: author_id, Length: 17860, dtype: object

In [117]:
# 독자 인덱스
train_key_sum['readers_id']

0        #0013c35c789f0936654ba06389ea75cb
1        #00222a855547061a7fd00e404cb96148
2        #002cedb579c73f021b86f8b12d8e35d8
3        #002f5cac6b2c802ec323d3d3424a71cf
4        #00339d09d3f964d88ae3e57a81803539
                       ...                
12017    #fff10b2ae2f6e5a121a0adea7062898d
12018    #fff243d06ee500cd9af1fdfe7886bac7
12019    #fff892683009fe46182af61b859d2413
12020    #fffca5f83f8b7145b3102aec2e0934dc
12021    #fffe67ecc0056dd26ae00511957c5a2b
Name: readers_id, Length: 12022, dtype: object

In [118]:
# 유사도 행렬들에서 가장 높은 순으로 정렬 시키기
keyword_sim_sorted_ind = reader_author_keyword_sim.argsort()[:,::-1]
print(keyword_sim_sorted_ind[:1])

[[ 3931   665 11045 ... 11396 11394     0]]


In [119]:
keyword_sim_sorted_ind

array([[ 3931,   665, 11045, ..., 11396, 11394,     0],
       [15065,  9256, 10300, ...,  8786,  8789,     0],
       [10279, 14202,   988, ...,  9191,  9190,     0],
       ...,
       [15568,  4347, 16362, ...,  9585,  9583,     0],
       [ 8398,  2005,  7482, ...,  9050,  9045,     0],
       [17035,  3931, 12196, ...,  9033,  9032,     0]], dtype=int64)

## reader_id로 author_id 찾아주는 함수

In [120]:
# reader_id를 넣고 author_id를 찾아주는 함수
def find_sim_author_with_reader(df_reader, df_author, sorted_ind, reader_name, top_n=10):
    reader_list = df_reader[df_reader['readers_id']==reader_name]
    
    re_article_index = reader_list.index.values
    similar_indexes = sorted_ind[re_article_index, :(top_n)]
    
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    

    return df_author.iloc[similar_indexes]

In [121]:
train_key_sum[train_key_sum['readers_id']=='#0013c35c789f0936654ba06389ea75cb']

,readers_id,keyword_list,morphs_list
0,#0013c35c789f0936654ba06389ea75cb,"['육아', '결혼', '시어머니', '육아', '결혼', '시어머니', '결혼',...","육아, 결혼, 시어머니, 육아, 결혼, 시어머니, 결혼, 육아, 시어머니, 시어머니..."


In [122]:
reader_author_similar_list = find_sim_author_with_reader(train_key_sum, author_key_sum, keyword_sim_sorted_ind, '#0013c35c789f0936654ba06389ea75cb', 10)
reader_author_similar_list[['author_id', 'keyword_list']]

[[ 3931   665 11045 17370  2068  4087 12804 15047 10931  4540]]


,author_id,keyword_list
3931,@dotkvk,"['육아', '결혼', '시어머니', '연애', '육아', '결혼', '시어머니',..."
665,@allnew,"['육아', '직장인', '결혼']"
11045,@morningyogurt,"['시어머니', '결혼', '가족', '결혼', '우울증', '에세이', '결혼',..."
17370,@yoonarch,['육아']
2068,@brunchphw0,"['엄마', '친구이야기', '생각', '결혼', '결혼생활', '결혼', '사랑'..."
4087,@dust-universe,"['결혼', '결혼식', '시어머니', '시댁', '며느리', '결혼']"
12804,@puressuny,"['인간관계', '육아', '에세이', '엄마', '결혼', '육아', '육아', ..."
15047,@suason88,"['시어머니', '가족', '문화']"
10931,@mongmong,"['레고프로포즈', '결혼', '꽁냥꽁냥']"
4540,@eunsimlee,['결혼']


# 지수가 짠 100개 추천 코드

In [ ]:
pop_recom = pd.read_csv("train_.csv")

In [ ]:
# test 예시
# cosine : 독자-작가 코사인 유사도 뽑은 한줄 array
# reader_cosine : 독자-독자 코사인 유사도 뽑은 한줄 array (test용으로 만들지 않앗어용)
# authors_id : 작가 list (코사인 유사도 컬럼)
cosine = np.arange(0,30)
authors_id = ["언", "니", "화", "이", "팅", "해", "용", "가", "나", "다", "라", "마", "바", "가", "나", "다", "라", "마", "바", "가", "나", "다", "라", "마", "바", "가", "나", "다", "라", "마"]
cosine

In [ ]:
## 코사인 유사도 받은 데이터 index 와 value 값 저장 해두기
# 독자-작가
idx = dict(enumerate(cosine)).keys()
value = dict(enumerate(cosine)).values()
author_data = pd.author_dataFrame({"authors":authors_id, "idx":list(idx), "value":list(value)})
author_data

# 독자-독자
idx = dict(enumerate(reader_cosine)).keys()
value = dict(enumerate(reader_cosine)).values()
reader_data = pd.author_dataFrame({"readers":readers_id, "idx":list(idx), "value":list(value)})
reader_data

In [ ]:
## 독자-작가 유사성을 통한 작가가 쓴 글 추천

def users_author_recom(author_data, reader_data): #author data를 10개 자른게 authors
    
    
    
    reader_author_similar_list = find_sim_author_with_reader(train_key_sum, author_key_sum, keyword_sim_sorted_ind, '#0013c35c789f0936654ba06389ea75cb', 10)
    reader_author_similar_list[['author_id', 'keyword_list']]
    
    
    # reader_id를 넣고 author_id를 찾아주는 함수
def find_sim_author_with_reader(df_reader, df_author, sorted_ind, reader_name, top_n=10):
    reader_list = df_reader[df_reader['readers_id']==reader_name]
    
    re_article_index = reader_list.index.values
    similar_indexes = sorted_ind[re_article_index, :(top_n)]
    
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    

    return df_author.iloc[similar_indexes]
    
    
    
    
    ## 독자-작가 유상성을 통한 작가가 쓴 글 추천
    recommend_df = pd.DataFrame(columns=["title", "author_id", "display_url"])

    for author in authors:
        author_recom = metadata[metadata.author_id == author].sort_values(by="reg_datetime", ascending=False)
        if len(author_recom) >= 5:
            recommend_df = pd.concat([recommend_df, author_recom[["title", "author_id", "display_url", "article_id"]].iloc[:5]])
        else:
            recommend_df = pd.concat([recommend_df, author_recom[["title", "author_id", "display_url", "article_id"]]])
    
    # 독자-작가 추천 후 추천한 글 제외하기 (겹치지 않기 위해)
    metadata = metadata[~metadata['article_id'].isin(list(recommend_df.article_id))]
    
    ## 독자-독자 유상성을 통한 작가가 쓴 글 추천
    # ["@bookdb", "@kohwang56", "@hannahajink", "@bryceandjuli", "@mijeongpark"] 지우시고 뒤에 코드 주석 빼고 넣으시면 됩니당    
    readers = ["@bookdb", "@kohwang56", "@hannahajink", "@bryceandjuli", "@mijeongpark"]#reader_data.sort_values(by="value", ascending=False).readers.iloc[:10]

    for reader in readers:
        reader_aritcle_ls=list(read[read.readers_id == reader].article_id)
        reader_recom = metadata[metadata['article_id'].isin(reader_aritcle_ls)].sort_values(by="reg_datetime", ascending=False)
        reader_recom = metadata[metadata.author_id == author].sort_values(by="reg_datetime", ascending=False)
        if len(reader_recom) >= 5:
            recommend_df = pd.concat([recommend_df, reader_recom[["title", "author_id", "display_url", "article_id"]].iloc[:5]])
        else:
            recommend_df = pd.concat([recommend_df, reader_recom[["title", "author_id", "display_url", "article_id"]]])
    
    # 독자-독자 추천 후 추천한 글 제외하기 (겹치지 않기 위해)
    metadata = metadata[~metadata['article_id'].isin(list(recommend_df.article_id))]
    
    
     ## 독자-작가/독자-독자 추천 시에 100이 안될 경우
    if len(recommend_df) == 100:
        return recommend_df
    else:
        count = 100-len(recommend_df)
        popular_recom_article_id = pop_recom.sort_values(by=["recent_weight", "popular_weight"], ascending=False).iloc[:count,4]
        popular_recom = pd.merge(left=popular_recom_article_id, right=metadata, left_on="article_id", right_on="article_id", how="left")
        
        pd.concat([recommend, popular_recom[["title", "author_id", "display_url", "article_id"]]])

    return recommend

In [ ]:
## 독자-작가 유사성을 통한 작가가 쓴 글 추천

def users_author_recom(author_data, reader_data):
    
    ## 독자-작가 유상성을 통한 작가가 쓴 글 추천
    # ["@bookdb", "@kohwang56", "@hannahajink", "@bryceandjuli", "@mijeongpark"] 지우시고 뒤에 코드 주석 빼고 넣으시면 됩니당
    authors = ["@bookdb", "@kohwang56", "@hannahajink", "@bryceandjuli", "@mijeongpark"] #author_data.sort_values(by="value", ascending=False).authors.iloc[:10]
    recommend_df = pd.DataFrame(columns=["title", "author_id", "display_url"])

    for author in authors:
        author_recom = metadata[metadata.author_id == author].sort_values(by="reg_datetime", ascending=False)
        if len(author_recom) >= 5:
            recommend_df = pd.concat([recommend_df, author_recom[["title", "author_id", "display_url", "article_id"]].iloc[:5]])
        else:
            recommend_df = pd.concat([recommend_df, author_recom[["title", "author_id", "display_url", "article_id"]]])
    
    # 독자-작가 추천 후 추천한 글 제외하기 (겹치지 않기 위해)
    metadata = metadata[~metadata['article_id'].isin(list(recommend_df.article_id))]
    
    ## 독자-독자 유상성을 통한 작가가 쓴 글 추천
    # ["@bookdb", "@kohwang56", "@hannahajink", "@bryceandjuli", "@mijeongpark"] 지우시고 뒤에 코드 주석 빼고 넣으시면 됩니당    
    readers = ["@bookdb", "@kohwang56", "@hannahajink", "@bryceandjuli", "@mijeongpark"]#reader_data.sort_values(by="value", ascending=False).readers.iloc[:10]

    for reader in readers:
        reader_aritcle_ls=list(read[read.readers_id == reader].article_id)
        reader_recom = metadata[metadata['article_id'].isin(reader_aritcle_ls)].sort_values(by="reg_datetime", ascending=False)
        reader_recom = metadata[metadata.author_id == author].sort_values(by="reg_datetime", ascending=False)
        if len(reader_recom) >= 5:
            recommend_df = pd.concat([recommend_df, reader_recom[["title", "author_id", "display_url", "article_id"]].iloc[:5]])
        else:
            recommend_df = pd.concat([recommend_df, reader_recom[["title", "author_id", "display_url", "article_id"]]])
    
    # 독자-독자 추천 후 추천한 글 제외하기 (겹치지 않기 위해)
    metadata = metadata[~metadata['article_id'].isin(list(recommend_df.article_id))]
    
    
     ## 독자-작가/독자-독자 추천 시에 100이 안될 경우
    if len(recommend_df) == 100:
        return recommend_df
    else:
        count = 100-len(recommend_df)
        popular_recom_article_id = pop_recom.sort_values(by=["recent_weight", "popular_weight"], ascending=False).iloc[:count,4]
        popular_recom = pd.merge(left=popular_recom_article_id, right=metadata, left_on="article_id", right_on="article_id", how="left")
        
        pd.concat([recommend, popular_recom[["title", "author_id", "display_url", "article_id"]]])

    return recommend